In [1]:
import torch
from torch import nn

### Домашнее задание

На основании функции по варианту необходимо:
1. Построить nn.Module, в котором вы определите `forward`и `my_forward_backward`
2. `forward` должен повторять функцию, выданную вам по варианту, `my_forward_backward` описывает проход по вычислителному графу, а также вычисление градиентов по этому графу с помощью backprop. Градиенты должны быть рассчитаны для параметров $w0, w1$, тензоры $x1, x2, x3$ считаются входными данными сети
3. Необходимо удалять неиспользуемые тензоры, как это делает Pytorch
4. Если какие-то узлы не нужны для вычисления результата, то вы не должны их вычислять в процессе backprop
5. Необходимо построить визуализацию вычислительного графа

Все материалы для выполнения задания можно найти в ноутбуках второго и третьего семинаров, а также в этом ноутбуке есть пример выполненного задания

In [ ]:
def v4():
    r1 = 3 * w0 + x3 * w1 + 1
    r2 = x2 ** 3 * w1 / w0
    r3 = w0 * (x3 + w1 ** 2)
    return (r1 - r3) * x3 * r2 / r1

In [2]:
def test_model_class(model_class):

    model = model_class()

    for _ in range(10):

        x1, x2, x3 = torch.rand(3)

        model.zero_grad()
        y_torch = model(x1, x2, x3)
        y_torch.backward()
        grad_torch = model.w.grad.clone()

        print(y_torch.clone().detach().numpy(), grad_torch.clone().numpy())

        model.zero_grad()
        with torch.no_grad():
            y_manual = model.your_forward_backward(x1, x2, x3)
        grad_manual = model.w.grad.clone()

        print(y_manual.numpy(), grad_manual.numpy())
        # print()

        assert torch.allclose(y_manual, y_torch, rtol=5e-05, atol=1e-7)
        assert torch.allclose(grad_manual, grad_torch, rtol=5e-05, atol=1e-7)

    print('Tests completed successfully!')

In [40]:
# Пример

class ExampleGraph(nn.Module):

    def __init__(self):
        super().__init__()
        self.w = nn.Parameter(torch.tensor([0.950, 0.288], dtype=torch.float32))

    def forward(self, x1, x2, x3):
        r1 = x3 * (x1 / x2 + self.w[1] / self.w[0])
        r2 = x1 * x2 * x3 / (self.w[0] + self.w[1])
        r3 = (self.w[0] * self.w[1]) ** 3.0
        return (r1 + r2 + r3) / (x1 * r1 + r2 + r3)

    def your_forward_backward(self, x1, x2, x3):
        w0 = self.w[0]
        w1 = self.w[1]

        # forward
        a0 = x1 / x2
        a1 = w1 / w0
        a2 = a0 + a1

        a0 = a1 = None

        a3 = x3 * a2 # r1

        a2 = None

        a4 = w0 + w1
        a5 = x1 * x2

        x2 = None

        a6 = a5 * x3

        a5 = None

        a7 = a6 / a4 # r2

        a8 = w0 * w1
        a9 = a8 ** 3 # r3
        a10 = a7 + a9 # r2 + r3

        a7 = a9 = None

        a11 = a10 + a3
        a12 = x1 * a3

        a3 = None

        a13 = a12 + a10

        a10 = a12 = None

        a14 = a11 / a13

        # backward
        da14 = 1.0
        da13 = da14 * ( - a11 / a13**2)

        a11 = None

        da12 = da13
        da11 = da14 * (1 / a13)

        a13 = da14 = None

        da10 = da11 + da13

        da13 = None

        da9 = da10

        da8 = da9 * 3 * a8 ** 2

        da9 = a8 = None

        da7 = da10

        da10 = None

        # da6 и da5 не нужны для вычисления итогового градиента

        da4 = da7 * ( - a6 / a4 ** 2)

        da7 = a6 = a4 = None

        da3 = da11 + da12 * x1

        da11 = da12 = x1 = None

        da2 = da3 * x3

        x3 = da3 = None

        da1 = da2

        da2 = None

        dw1 = da8 * w0 + da4 + da1 / w0
        dw0 = da8 * w1 + da1 * (- w1 / w0 ** 2) + da4

        da1 = da4 = da8 = w0 = w1 = None

        self.w.grad = torch.stack([dw0, dw1])

        return a14

In [3]:
class Graph(nn.Module):
    def __init__(self):
        super().__init__()
        self.w = nn.Parameter(torch.tensor([0.950, 0.288], dtype=torch.float32))
    
    def forward(self, x1, x2, x3):
        r1 = 3 * self.w[0] + x3 * self.w[1] + 1
        r2 = x2 ** 3 * self.w[1] / self.w[0]
        r3 = self.w[0] * (x3 + self.w[1] ** 2)
        return (r1 - r3) * x3 * r2 / r1
    
    def your_forward_backward(self, x1, x2, x3):
        w0 = self.w[0]
        w1 = self.w[1]
        
        # forward
        a0 = x2**3
        
        a1 = a0 * w1
        a2 = w0 * 3
        a3 = w1 * x3
        a4 = w1 ** 2
        
        a5 = a1 / w0 # r2
        a6 = a2 + a3
        a7 = a4 + x3
        
        a8 = a6 + 1  # r1
        a9 = w0 * a7 # r3
        
        a10 = a8 - a9
        
        a11 = a10 * x3
        
        a12 = a11 * a5
        
        a13 = a12 / a8
        
        # backward
        
        da13 = 1.0
        
        da12 = da13 / a8
        
        da11 = da12 * a5
        
        da10 = da11 * x3
        
        da9 = da10 * (-1.0)
        
        da8 = da10 + da13 * (-a12 / a8 ** 2)
        
        da7 = da9 * w0
        
        da6 = da8
        
        da5 = da12 * a11 
        
        da4 = da7  
        
        da3 = da6
        
        da2 = da6
        
        da1 = da5 / w0
        
        da0 = None
        
        dw1 = da1 * a0 + da3 * x3 + da4 * 2 * w1
        
        #da1 = da3 = da4 = a0 = x3 = w1 = None
        
        dw0 = da5 * ( -a1 / w0 ** 2) + da2 * 3 + da9 * a7 
        
        #da5 = da2 = da9 = a1 = a7 = w0 = None
        
        self.w.grad = torch.stack([dw0, dw1])
        
        return a13

In [4]:
# Здесь тестируется построенная вами модель
test_model_class(Graph)

5.04462e-07 [-5.4305366e-07  1.6777796e-06]
5.04462e-07 [-5.4305360e-07  1.6777796e-06]
0.0004354608 [-0.0005053   0.00146994]
0.0004354608 [-0.0005053   0.00146994]
0.10711705 [-0.12429416  0.36158073]
0.10711705 [-0.12429415  0.36158073]
0.061641082 [-0.07076722  0.20740867]
0.061641082 [-0.07076722  0.20740867]
0.20283829 [-0.23301917  0.6826328 ]
0.20283829 [-0.23301917  0.6826328 ]
0.0006178984 [-0.00065484  0.00205559]
0.0006178984 [-0.00065484  0.00205559]
0.084349915 [-0.09182571  0.28075683]
0.084349915 [-0.09182572  0.28075683]
0.063442156 [-0.06850059  0.21103042]
0.063442156 [-0.06850059  0.21103042]
0.0075654844 [-0.0083219   0.02521407]
0.0075654844 [-0.0083219   0.02521407]
0.007493767 [-0.00821607  0.02496355]
0.007493767 [-0.00821607  0.02496355]
Tests completed successfully!


In [3]:
model = Graph()

x1 = torch.randn(5)
x2 = torch.randn(5)
x3 = torch.randn(5)

torch.onnx.export(
    model,
    (x1, x2, x3),
    'graph.onnx',
    opset_version=13,
    export_params=True,
    do_constant_folding=False,
    input_names=['x1', 'x2', 'x3'],
    output_names=['y'],
    )

In [ ]:
# А сюда вы должны вставить визуализацию графа

![title](graph.png)